In [38]:
import tarfile
import os
import numpy as np
import pandas as pd
import h5py
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt

In [ ]:
#User name and Key have been set in /home/user/.kaggle/kaggle.json and rules have been accepted 
! sudo kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

In [ ]:
#Unzipping file
for file in os.listdir('./'):
    if (file.endswith("tar.gz")):
        tar = tarfile.open(file)
        tar.extractall()
        tar.close()

In [ ]:
#From csv to a numpy array
pictures = np.genfromtxt('./fer2013/fer2013.csv' , delimiter = ','  , dtype=  None)
pictures.shape

In [ ]:
#Transforming data to float, so that, we could train our model

x_training , y_training, x_test , y_test = np.ones((1,2304)) , np.ones((1,1)) , np.ones((1,2304)) , np.ones((1,1))
print(x_training.shape,  y_training.shape , x_test.shape , y_test.shape)
converter_pic = lambda z : np.fromstring( z , dtype = float, sep = " ")

temp = np.fromstring(pictures[20][1] , dtype = float , sep = ' ')
temp = temp.reshape(48,48)
plt.imshow(temp)
print(pictures[20][2])

#Data is already splitted into training and test sets , so , we just have load them.
for picture in pictures:
    if picture[0] == "emotion":
        pass
    elif picture[2] == "Training":
        x_training = np.r_[x_training, converter_pic(picture[1]).reshape(1,-1)]
        y_training = np.append(y_training, int(picture[0]))
    else:
        x_test = np.r_[x_test, converter_pic(picture[1]).reshape(1,-1)]
        y_test = np.append(y_test, int(picture[0]))
    
print(x_training , y_training)
x_training.shape,  y_training.shape , x_test.shape , y_test.shape


In [ ]:
#Writing data on disk

with h5py.File('./data_emotion.h5' , 'w') as hdf:
    hdf.create_dataset('x_test' , data = x_test)
    hdf.create_dataset('y_test' , data = y_test)
    hdf.create_dataset('x_training' , data = x_training)
    hdf.create_dataset('y_training' , data = y_training)

In [75]:
#This is just for test that everything is ok.

with h5py.File('./data_emotion.h5' , 'r') as hdf:
    ls = list(hdf.keys())
    not_chunked_x_training = np.array(hdf.get('x_training'))
    not_chunked_y_training = np.array(hdf.get('y_training'))
    not_chunked_x_test = np.array(hdf.get('x_test'))
    not_chunked_y_test = np.array(hdf.get('y_test'))
    
    print("list", ls)

('list', [u'x_test', u'x_training', u'y_test', u'y_training'])


In [76]:
not_chunked_x_training.shape, not_chunked_y_training.shape , not_chunked_x_test.shape , not_chunked_y_test.shape

((28710, 2304), (28710,), (7179, 2304), (7179,))

In [122]:
#Let's simulate that we run out of memory so we have to train our model by chunks

model = SGDClassifier(loss='log', penalty="l2")
array_classes = np.array([0,1,2,3,4,5,6])
sliding_window = 10 
total_slides = (28710 / sliding_window) - 1
#Open h5 file
i = 1
with h5py.File('./data_emotion.h5' , 'r') as hdf:
    for idx in range(total_slides):
        temp_x = np.array(hdf["x_training"][idx*sliding_window:(idx+1)*sliding_window , :])
        temp_y = np.array(hdf["y_training"][idx*sliding_window:(idx+1)*sliding_window]).reshape(-1,1)
        i = i + 1
        model.partial_fit(temp_x, temp_y, classes = array_classes)
print(i)


TypeError: partial_fit() got an unexpected keyword argument 'shuffle'

In [121]:
print(model.predict(not_chunked_x_test))
model.score(not_chunked_x_test,not_chunked_y_test)

[3 6 3 ... 3 3 3]


0.2969772948878674

In [113]:
with h5py.File('./data_emotion_weights.h5' , 'w') as hdf:
    hdf.create_dataset('weights' , data = model.coef_)

In [114]:
matrix1 = np.random.random(size = (100,1000))
data_file_example = h5py.File("example_data_hdf5" , "w")


IOError: Unable to create file (unable to truncate a file which is already open)

In [ ]:
dset = data_file_example.create_dataset("chunked", matrix1 , chunks=(10, 1000))


In [ ]:
with h5py.File('./example_data_hdf5' , 'r') as hdf:
    ls = list(hdf.keys())
    print(ls)

In [ ]:
! pwd

In [ ]:
clf = linear_model.SGDClassifier()

In [ ]:
x = pd.read_csv("./fer2013.csv")


In [ ]:
x.head()